In [82]:
import os
import requests
import pandas as pd
from dotenv import load_dotenv
import datetime as dt
import alpaca_trade_api as tradeapi
import json
from alpaca_trade_api.rest import REST, TimeFrame
from libs.drew_lib import *
load_dotenv()

True

In [2]:
alpaca_key = os.getenv("ALPACA_API_KEY")
alpaca_secret_key = os.getenv("ALPACA_SECRET_KEY")

tradeapi = REST(alpaca_key, alpaca_secret_key)

In [3]:
# def get_daily_trade_data(ticker, start, end, tradeapi): 
#     ticker_df = tradeapi.get_bars(
#         ticker,
#         TimeFrame.Day,
#         start,
#         end
#     ).df
#     ticker_df['ticker'] = ticker

#     return ticker_df

In [4]:
# ticker_list = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')['Ticker'].to_list()

# def make_tickers_df(ticker_list, start_date_str, end_date_str, tradeapi):
#     ticker_dfs_list = [get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list]
#     tickers_df = pd.concat(ticker_dfs_list, axis=0, join='outer')
#     tickers_df.index = tickers_df.index.date
#     tickers_df = tickers_df[['ticker','close','volume']]
#     return tickers_df


In [5]:
# def make_tickers_dict(ticker_list, start_date_str, end_date_str, tradeapi):
#     return {ticker: get_daily_trade_data(ticker, start_date_str, end_date_str, tradeapi) for ticker in ticker_list}

## News Article Vader Analyzer

In [3]:
from libs.drew_lib import *

from newsapi import NewsApiClient
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer

nltk.download('vader_lexicon')

analyzer = SentimentIntensityAnalyzer()

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     C:\Users\altma\AppData\Roaming\nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [4]:
api_key = os.getenv("NEWSAPI_KEY")
newsapi = NewsApiClient(api_key=api_key)
print(type(api_key))

<class 'str'>


In [8]:
# company_df = pd.read_csv('./Data/Cleaned_Data/Ticker_library.csv')
# company_df = company_df.rename(columns={'Company Name': 'Company_Name'})
# company_dict = dict(zip(company_df.Company_Name, company_df.Ticker))
# company_list = list(company_dict.keys())
# tickers_list = company_dict.values()
# ls = [type(item) for item in company_list]
# print(ls)
# tickers_list


In [9]:
def vader_analyzer(df):
    analyzer = SentimentIntensityAnalyzer()
    df['compound'] = [analyzer.polarity_scores(x)['compound'] for x in df['text']]
    df['neg'] = [analyzer.polarity_scores(x)['neg'] for x in df['text']]
    df['neu'] = [analyzer.polarity_scores(x)['neu'] for x in df['text']]
    df['pos'] = [analyzer.polarity_scores(x)['pos'] for x in df['text']]

    df['date'] = pd.to_datetime(
    df['date'],
    infer_datetime_format = True,
    utc = True    
    )
    df['date'] = df['date'].dt.date
    
    return df



In [ ]:
news_df_list = [form_df(company) for company in company_list]
news_df = pd.concat(news_df_list, axis=0, join='outer')
vader_news_df = vader_analyzer(news_df)
news_sent_avg = vader_news_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
news_sent_avg = news_sent_avg[['date','ticker','pos','neg','neu','compound']]
news_sent_avg

#news_sent_avg.to_csv('../Notebooks/Data/Cleaned_Data/news_sentiment.csv')

In [ ]:
# news_df_list = [form_df(company) for company in company_list]
# vader_news_df_list = [daily_sentiment(df) for df in news_df_list]
# vader_news_df = pd.concat(vader_news_df_list, axis=0, join='outer')
# vader_news_df.head()

In [11]:
# perform sentiment analysis and find average sentiment by date

def daily_sentiment(df):
    vader_df = vader_analyzer(df)
    vader_df = vader_df.groupby(['ticker','date'])['ticker','pos','neg','neu','compound'].mean().reset_index()
    vader_df = vader_df[['date','ticker','pos','neg','neu','compound']]
    return vader_df

## Helper Functions for PuttingItTogether

In [23]:
def return_bin(df):

    bins = [-1000, -0.08, -0.03, -0.0000001, 0.0000001, 0.03, 0.08, 1000]
    labels = ['large loss','medium loss','small loss','no gain/loss','small gain','medium gain','large gain']
    
    df['returns'] = df['close'].pct_change()
    df['return_bin'] = pd.cut(df['returns'], bins=bins, labels=labels)

    return df

## Data prep for model

In [81]:
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder

import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout

In [22]:
# ticker_list = ['AAPL', 'NFLX', 'TSLA']
# three_stocks = make_tickers_df(ticker_list, '2012-06-01', '2022-06-01', tradeapi)
# three_stocks.reset_index()#.rename(columns={'index':'date','symbol':'ticker'},inplace=True)
# aapl_df = three_stocks[three_stocks['symbol']=='AAPL']
# aapl_df
from libs.sec import *


In [40]:
uber_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/uber.csv')
abnb_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/airbnb.csv')
sec_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/sec_sentiment.csv')
news_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/news_sentiment.csv')
stock_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/stock_data.csv')


In [ ]:
sec_df.rename(columns={'periodOfReport':'date'},inplace=True)
sec_df['date'] = pd.to_datetime(sec_df['date'])
abnb_sec_df = sec_df[sec_df['ticker']=='ABNB']
uber_sec_df = sec_df[sec_df['ticker']=='UBER']
uber_sec_df.drop(columns=['Unnamed: 0'],inplace=True)
abnb_sec_df.drop(columns=['Unnamed: 0'],inplace=True)
abnb_sec_df


In [56]:
# Reindex to the daily index, forward fill.
def ffill_dates(df, start_date, end_date, date_col):
    idx_daily = pd.date_range(start=start_date, end=end_date, freq='D')
    df = df.set_index(date_col).reindex(idx_daily, method='ffill').reset_index().rename(columns={'index': 'date'})
    return df

In [ ]:
abnb_sec_df = ffill_dates(abnb_sec_df, '2020-12-31', '2022-06-01', 'date')
abnb_sec_df

In [69]:
abnb_sec_df.set_index('date',inplace=True)

In [ ]:
uber_sec_df = ffill_dates(uber_sec_df, '2019-03-31', '2022-06-01', 'date')
uber_sec_df

In [70]:
uber_sec_df.set_index('date',inplace=True)

In [ ]:
uber_df.rename(columns={'Unnamed: 0': 'date'},inplace=True)
uber_df['date'] = pd.to_datetime(uber_df['date'])
uber_df = return_bin(uber_df)
uber_df = uber_df[uber_df['return_bin'].notna()]
uber_df.set_index('date',inplace=True)

In [80]:
uber_data = pd.concat([uber_sec_df,uber_df],axis=1,join='outer')
uber_data['return_bin'] = uber_data['return_bin'].fillna('no gain/loss')
uber_data['returns'] = uber_data['returns'].fillna(0)
uber_data[['stockmarket_compound_sentiment','stockmarket_positive_sentiment','stockmarket_neutral_sentiment','stockmarket_negative_sentiment','close']] = uber_data[['stockmarket_compound_sentiment','stockmarket_positive_sentiment','stockmarket_neutral_sentiment','stockmarket_negative_sentiment','close']].fillna(method='bfill')
uber_data['volume'] = uber_data['volume'].fillna(uber_data['volume'].mean())
uber_data = uber_data.drop(columns='ticker').rename(
    columns={'pos':'sec_positive_sentiment',
    'neg':'sec_negative_sentiment',
    'neu':'sec_neutral_sentiment',
    'compound':'sec_compound_sentiment'}
)
uber_data

,sec_positive_sentiment,sec_negative_sentiment,sec_neutral_sentiment,sec_compound_sentiment,stockmarket_compound_sentiment,stockmarket_positive_sentiment,stockmarket_neutral_sentiment,stockmarket_negative_sentiment,close,volume,returns,return_bin
date,,,,,,,,,,,,
2019-03-31,0.111,0.042,0.846,1.0,0.0258,0.151,0.7085,0.1405,139.25,6.719166e+06,0.000000,no gain/loss
2019-04-01,0.111,0.042,0.846,1.0,0.0258,0.151,0.7085,0.1405,139.25,6.719166e+06,0.000000,no gain/loss
2019-04-02,0.111,0.042,0.846,1.0,0.0258,0.151,0.7085,0.1405,139.25,6.719166e+06,0.000000,no gain/loss
2019-04-03,0.111,0.042,0.846,1.0,0.0258,0.151,0.7085,0.1405,139.25,6.719166e+06,0.000000,no gain/loss
2019-04-04,0.111,0.042,0.846,1.0,0.0258,0.151,0.7085,0.1405,139.25,6.719166e+06,0.000000,no gain/loss
...,...,...,...,...,...,...,...,...,...,...,...,...
2022-05-28,0.126,0.040,0.833,1.0,0.4404,0.081,0.8630,0.0560,120.87,6.719166e+06,0.000000,no gain/loss
2022-05-29,0.126,0.040,0.833,1.0,0.4404,0.081,0.8630,0.0560,120.87,6.719166e+06,0.000000,no gain/loss
2022-05-30,0.126,0.040,0.833,1.0,0.4404,0.081,0.8630,0.0560,120.87,6.719166e+06,0.000000,no gain/loss


In [20]:
# abnb_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/stock_data.csv',infer_datetime_format=True,parse_dates=True)
# abnb_df = abnb_df[abnb_df['ticker']=='ABNB']
# abnb_df = return_bin(abnb_df)
# abnb_df = abnb_df[abnb_df['return_bin'].notna()]
# abnb_df

,date,ticker,close,volume,returns,return_bin
5684,2020-12-11,ABNB,139.25,26981241,-0.037731,medium loss
5685,2020-12-14,ABNB,130.00,16966090,-0.066427,medium loss
5686,2020-12-15,ABNB,124.80,10914466,-0.040000,medium loss
5687,2020-12-16,ABNB,137.99,20410248,0.105689,large gain
5688,2020-12-17,ABNB,147.05,15054741,0.065657,medium gain
...,...,...,...,...,...,...
6049,2022-05-25,ABNB,110.40,6170197,0.039157,medium gain
6050,2022-05-26,ABNB,114.30,6884206,0.035326,medium gain
6051,2022-05-27,ABNB,120.50,7375113,0.054243,medium gain
6052,2022-05-31,ABNB,120.87,9089352,0.003071,small gain


In [44]:
#nflx_df = stock_picker('NFLX')
stock_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/stock_data.csv', infer_datetime_format=True, parse_dates=True)
stock_df['ticker'].value_counts()
#abnb_df = stock_df[stock_df['ticker']=='ABNB']
#abnb_df = return_bin(abnb_df)
#abnb_df = abnb_df[abnb_df['return_bin'].notna()]
#abnb_df.head()

NFLX    1637
FB      1637
MCHP    1637
FANG    1637
MRO     1637
DVN     1637
SPWR    1637
REGI    1637
MTRX    1637
BLK     1637
PYPL    1637
MELI    1637
UBER     772
ABNB     371
SOFI     254
Name: ticker, dtype: int64

In [43]:
sec_df = pd.read_csv('../Notebooks/Data/Cleaned_Data/sec_all_data.csv')
sec_df = sec_df[sec_df['ticker']=='ABNB']
sec_df = sec_df[['periodOfReport', 'ticker', 'Business', 'Risk Factors', 'Management’s Discussion and Analysis of Financial Condition and Results of Operations']]#.agg('-'.join, axis=1)
sec_df['text'] = sec_df[['Business', 'Risk Factors', 'Management’s Discussion and Analysis of Financial Condition and Results of Operations']].astype(str).apply(lambda x: ''.join(x), axis=1)
#sec_df = sec_df[['periodOfReport', 'ticker', 'text']]
sec_df.drop(columns=['Business', 'Risk Factors', 'Management’s Discussion and Analysis of Financial Condition and Results of Operations'],inplace=True)
sec_df


,periodOfReport,ticker,text
100,2022-03-31,ABNB,nanundefined Item 2. Management&#8217;s Discus...
101,2021-09-30,ABNB,nanundefined Item 2. Management&#8217;s Discus...
102,2021-06-30,ABNB,nanundefined Item 2. Management&#8217;s Discus...
103,2021-03-31,ABNB,nanundefined Item 2. Management&#8217;s Discus...
403,2021-12-31,ABNB,Item 1. Business Overview We are a community ...
404,2020-12-31,ABNB,Item 1. Business Overview Airbnb is a communi...


In [126]:
X = uber_data.copy()
X = uber_data.drop(columns=['return_bin'])
y = uber_data['return_bin'].values
y = y.reshape(-1, 1)

In [127]:
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [128]:
X_scaler = StandardScaler()
X_scaler.fit(X_train)

X_train_scaled = X_scaler.transform(X_train)
X_test_scaled = X_scaler.transform(X_test)

In [129]:
enc = OneHotEncoder()
enc.fit(y_train)

y_train_enc = enc.transform(y_train).toarray()
y_test_enc = enc.transform(y_test).toarray()

In [130]:
model = Sequential()

number_units = 30
dropout_fraction = 0.2

In [131]:
# Layer 1
model.add(LSTM(
    units=number_units,
    return_sequences=True,
    input_shape=(X_train_scaled.shape[1], 1))
    )
model.add(Dropout(dropout_fraction))

# Layer 2
model.add(LSTM(units=number_units, return_sequences=True))
model.add(Dropout(dropout_fraction))

# Layer 3
model.add(LSTM(units=number_units))
model.add(Dropout(dropout_fraction))

# Output layer
model.add(Dense(1))

# Compile the model
model.compile(optimizer="adam", loss="mean_squared_error",metrics=["accuracy"])

# Train the model
model.fit(X_train_scaled, y_train_enc, epochs=100, shuffle=False, batch_size=90, verbose=1)

# Evaluate the model
model_loss = model.evaluate(X_test_scaled, y_test_enc, verbose=2)
print(f"Normal Neural Network - Loss: {model_loss}")


# Make predictions using the testing data X_test
predicted = model.predict(X_test_scaled)
#predicted = enc.inverse_transform(predicted).flatten().tolist()

Epoch 1/100
10/10 [==============================] - 2s 9ms/step - loss: 0.1299 - accuracy: 0.8571
Epoch 2/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1240 - accuracy: 0.8571
Epoch 3/100
10/10 [==============================] - 0s 10ms/step - loss: 0.1232 - accuracy: 0.8571
Epoch 4/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1228 - accuracy: 0.8571
Epoch 5/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1228 - accuracy: 0.8571
Epoch 6/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1227 - accuracy: 0.8571
Epoch 7/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1227 - accuracy: 0.8571
Epoch 8/100
10/10 [==============================] - 0s 9ms/step - loss: 0.1227 - accuracy: 0.8571
Epoch 9/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1227 - accuracy: 0.8571
Epoch 10/100
10/10 [==============================] - 0s 8ms/step - loss: 0.1227 - accuracy: 0.8571
Epoch 11

In [133]:
#predicted = enc.inverse_transform(predicted).flatten().tolist()
results = pd.DataFrame({
    "Actual": y_test_enc.values,
    "Predicted": predicted
})
results.head(10)

AttributeError: 'numpy.ndarray' object has no attribute 'values'